In [1]:
from llm_manager import (
    SequenceClassifierOutputOverRanges,
    EmbeddingBasedClassifier,
    ClassifierBase,
    ID2LABEL,
    LABEL2ID,
    MODEL_NAME,
)
from dataset_manager import ROOT, MovieLensManager, INPUT_EMBEDS_REPLACE_KGE_DIMENSION
from graph_representation_generator import GraphRepresentationGenerator
from typing import Optional, Union, Tuple
import os

import torch
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers import BertForSequenceClassification

# Customize Input Embeds Replace Classifier
In our first experiments we noticed that replacing the placeholders of the input embeds with the KGEs breakes up the gradient propagation all the way back. To make it even, we will freeze this entire step so that only the attention headers are trained. 

In [2]:
class InputEmbedsReplaceFrozenBertForSequenceClassification(
    BertForSequenceClassification
):
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        graph_embeddings: Optional[torch.Tensor] = None,
        semantic_positional_encoding: Optional[torch.Tensor] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutputOverRanges]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )
        if inputs_embeds is None:
            # CHANGES: we freeze the gradients when producing input embeddings and detach them before passing them on just to be save
            with torch.no_grad():
                inputs_embeds = self.bert.embeddings(input_ids).detach()
            assert isinstance(inputs_embeds, torch.Tensor)
        if graph_embeddings is not None and len(graph_embeddings) > 0:
            if attention_mask is not None:
                mask = (
                    (
                        (attention_mask.to(self.device).sum(dim=1) - 1)
                        .unsqueeze(1)
                        .repeat((1, 2))
                        - torch.tensor([3, 1], device=self.device)
                    )
                    .unsqueeze(2)
                    .repeat((1, 1, self.config.hidden_size))
                )  # basically a mask finding the last positions between the sep tokens (reshaped so they can be used in scatter)
                inputs_embeds = inputs_embeds.to(
                    self.device
                ).scatter(
                    1, mask.to(self.device), graph_embeddings.to(self.device)
                )  # replace the input embeds at the place holder positions with the KGEs.
        outputs = self.bert(
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )  # feed forward the input embeds to the attention model

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (
                    labels.dtype == torch.long or labels.dtype == torch.int
                ):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutputOverRanges(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            semantic_positional_encoding=semantic_positional_encoding,
        )

Next we define the new Classifier the same way the original was defined, with the exception of the model we are loading, which is the new frozen model.

In [3]:
class InputEmbedsReplaceFrozenClassifier(EmbeddingBasedClassifier):
    def __init__(
        self,
        kge_manager,
        get_embedding_cb,
        root_path,
        model_name=MODEL_NAME,
        model_max_length=256,
        false_ratio=1.0,
        force_recompute=False,
    ) -> None:
        training_path = f"{root_path}/training"
        model_path = f"{training_path}/best"

        if os.path.exists(model_path) and not force_recompute:
            model = (
                InputEmbedsReplaceFrozenBertForSequenceClassification.from_pretrained(
                    model_path,
                    num_labels=2,
                    id2label=ID2LABEL,
                    label2id=LABEL2ID,
                )
            )
        else:
            model = (
                InputEmbedsReplaceFrozenBertForSequenceClassification.from_pretrained(
                    model_name, num_labels=2, id2label=ID2LABEL, label2id=LABEL2ID
                )
            )
        assert isinstance(model, BertForSequenceClassification)
        super().__init__(
            kge_manager,
            get_embedding_cb,
            root_path,
            model,
            model_name,
            model_max_length,
            false_ratio,
            force_recompute,
        )

    def plot_training_loss_and_accuracy(self):
        model_type = "Input Embeds Replace"
        self._plot_training_loss_and_accuracy(model_type)

Now comes the whole training procedure (see training_models.ipynb)

In [4]:
kg_manager = MovieLensManager()
EPOCHS = 20
BATCH_SIZE = 256
graph_representation_generator_input_embeds_replace = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
)
input_embeds_replace_embeddings = (
    graph_representation_generator_input_embeds_replace.get_saved_embeddings(
        "input_embeds_replace"
    )
)
save = False
if input_embeds_replace_embeddings is None:
    input_embeds_replace_embeddings = (
        graph_representation_generator_input_embeds_replace.generate_embeddings(
            kg_manager.llm_df
        )
    )
    save = True
kg_manager.append_input_embeds_replace_graph_embeddings(
    input_embeds_replace_embeddings, save=save
)

loading pretrained model
Device: 'cuda'


In [5]:
INPUT_EMBEDS_REPLACE_FROZEN_ROOT = (
    f"{ROOT}/llm/input_embeds_replace_frozen"  # make sure the dir exists
)
if not os.path.exists(INPUT_EMBEDS_REPLACE_FROZEN_ROOT):
    os.makedirs(INPUT_EMBEDS_REPLACE_FROZEN_ROOT)
input_embeds_replace_frozen_bert_classifier = InputEmbedsReplaceFrozenClassifier(
    kg_manager,
    graph_representation_generator_input_embeds_replace.get_embedding,
    root_path=INPUT_EMBEDS_REPLACE_FROZEN_ROOT,
)
dataset_input_embeds_replace = (
    kg_manager.generate_input_embeds_replace_embedding_dataset(
        input_embeds_replace_frozen_bert_classifier.tokenizer.sep_token,
        input_embeds_replace_frozen_bert_classifier.tokenizer.pad_token,
        input_embeds_replace_frozen_bert_classifier.tokenize_function,
        force_recompute=True,
    )
)
input_embeds_replace_frozen_bert_classifier.train_model_on_data(
    dataset_input_embeds_replace, epochs=EPOCHS, batch_size=BATCH_SIZE
)
# now we don't have to generate new non-existing edges, because they were generated in the beginning anyways.
input_embeds_replace_frozen_bert_classifier = InputEmbedsReplaceFrozenClassifier(
    kg_manager,
    graph_representation_generator_input_embeds_replace.get_embedding,
    root_path=INPUT_EMBEDS_REPLACE_FROZEN_ROOT,
    false_ratio=-1.0,  # init with false_ratio of -1 so no new false edges are produced on the fly
)
input_embeds_replace_frozen_df = (
    input_embeds_replace_frozen_bert_classifier.forward_dataset_and_save_outputs(
        dataset_input_embeds_replace,
        kg_manager.get_vanilla_tokens_as_df,
        epochs=1,
        batch_size=BATCH_SIZE,
        force_recompute=False,
    )
)

Some weights of InputEmbedsReplaceFrozenBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/56469 [00:00<?, ? examples/s]

Map:   0%|          | 0/34284 [00:00<?, ? examples/s]

Map:   0%|          | 0/34284 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/56469 [00:00<?, ? examples/s]

...

{'eval_loss': 0.27347999811172485, 'eval_accuracy': 0.8857192859642982, 'eval_runtime': 30.174, 'eval_samples_per_second': 1136.208, 'eval_steps_per_second': 4.441, 'epoch': 19.0}
{'loss': 0.3083, 'grad_norm': 0.7870933413505554, 'learning_rate': 2.806122448979592e-06, 'epoch': 19.0}
{'loss': 0.3131, 'grad_norm': 0.7621570825576782, 'learning_rate': 2.6785714285714285e-06, 'epoch': 19.05}
{'loss': 0.298, 'grad_norm': 0.9138600826263428, 'learning_rate': 2.5510204081632653e-06, 'epoch': 19.1}
{'loss': 0.2795, 'grad_norm': 0.8356846570968628, 'learning_rate': 2.423469387755102e-06, 'epoch': 19.14}
{'loss': 0.2872, 'grad_norm': 0.7734460830688477, 'learning_rate': 2.295918367346939e-06, 'epoch': 19.19}
{'loss': 0.3182, 'grad_norm': 0.8826451897621155, 'learning_rate': 2.1683673469387757e-06, 'epoch': 19.23}
{'loss': 0.302, 'grad_norm': 0.7545251846313477, 'learning_rate': 2.040816326530612e-06, 'epoch': 19.28}
{'loss': 0.2917, 'grad_norm': 0.9485779404640198, 'learning_rate': 1.9132653061

  0%|          | 0/134 [00:00<?, ?it/s]

{'eval_loss': 0.27520960569381714, 'eval_accuracy': 0.8859234628398086, 'eval_runtime': 30.26, 'eval_samples_per_second': 1132.981, 'eval_steps_per_second': 4.428, 'epoch': 20.0}
{'train_runtime': 9341.9345, 'train_samples_per_second': 120.894, 'train_steps_per_second': 0.473, 'train_loss': 0.3462586749732764, 'epoch': 20.0}
./data/llm/input_embeds_replace_frozen/attentions.npy ./data/llm/input_embeds_replace_frozen/hidden_states.npy ./data/llm/input_embeds_replace_frozen/tokens.csv
train Forward Epoch 1 from 1


BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


test Forward Epoch 1 from 1
val Forward Epoch 1 from 1


# Post-Training
After Training, we load all attentions and hidden states from before and append the new fields to the dataset.

In [6]:
vanilla_df = ClassifierBase.read_forward_dataset("./data/llm/vanilla")
prompt_df = ClassifierBase.read_forward_dataset("./data/llm/prompt")
input_embeds_replace_df = ClassifierBase.read_forward_dataset(
    "./data/llm/input_embeds_replace"
)

In [7]:
kg_manager.llm_df

,source_id,target_id,id_x,id_y,prompt_feature_title,prompt_feature_genres,labels,split,prompt,gnn_feature_(no genres listed),...,gnn_feature_IMAX,gnn_feature_Musical,gnn_feature_Mystery,gnn_feature_Romance,gnn_feature_Sci-Fi,gnn_feature_Thriller,gnn_feature_War,gnn_feature_Western,input_embeds_replace_source_embedding,input_embeds_replace_target_embedding
0,0,5,0.0,5.0,Heat (1995),"['Action', 'Crime', 'Thriller']",1,train,"0[SEP]5[SEP]Heat (1995)[SEP]['Action', 'Crime'...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.6693655252456665, 0.4152011275291443, -0.31...","[0.5493870973587036, -0.5692430734634399, -0.6..."
1,0,89,0.0,89.0,Bottle Rocket (1996),"['Adventure', 'Comedy', 'Crime', 'Romance']",1,train,0[SEP]89[SEP]Bottle Rocket (1996)[SEP]['Advent...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.6030164957046509, 0.37829291820526123, -0.3...","[0.08589676022529602, -0.4689454734325409, 0.0..."
2,0,97,0.0,97.0,Braveheart (1995),"['Action', 'Drama', 'War']",1,train,"0[SEP]97[SEP]Braveheart (1995)[SEP]['Action', ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.6387443542480469, 0.3706628084182739, -0.32...","[0.04940161854028702, -0.47019216418266296, -0..."
3,0,184,0.0,184.0,Billy Madison (1995),['Comedy'],1,train,0[SEP]184[SEP]Billy Madison (1995)[SEP]['Comedy'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.49271589517593384, 0.4562503397464752, -0.2...","[0.6636490821838379, -0.3091672956943512, 0.01..."
4,0,224,0.0,224.0,Star Wars: Episode IV - A New Hope (1977),"['Action', 'Adventure', 'Sci-Fi']",1,train,0[SEP]224[SEP]Star Wars: Episode IV - A New Ho...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.6498588919639587, 0.43191879987716675, -0.3...","[-0.08320911973714828, -0.060460180044174194, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125032,79,8846,NaN,NaN,Lovesick (2014),"['Comedy', 'Romance']",0,val,"79[SEP]8846[SEP]Lovesick (2014)[SEP]['Comedy',...",0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[-0.5735506415367126, -0.044633716344833374, -...","[0.47228026390075684, -0.911340594291687, 0.00..."
125033,300,2595,NaN,NaN,Dersu Uzala (1975),"['Adventure', 'Drama']",0,val,300[SEP]2595[SEP]Dersu Uzala (1975)[SEP]['Adve...,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.006543345749378204, -0.3807082176208496, 0....","[0.36955565214157104, -0.21707147359848022, -0..."
125034,321,59,NaN,NaN,Lawnmower Man 2: Beyond Cyberspace (1996),"['Action', 'Sci-Fi', 'Thriller']",0,val,321[SEP]59[SEP]Lawnmower Man 2: Beyond Cybersp...,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,"[-0.3852957785129547, 0.02625414729118347, -0....","[0.38845473527908325, -0.27163609862327576, -0..."
125035,388,2639,NaN,NaN,All the Vermeers in New York (1990),"['Comedy', 'Drama', 'Romance']",0,val,388[SEP]2639[SEP]All the Vermeers in New York ...,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[0.26095038652420044, 0.08665543794631958, -0....","[1.6701771020889282, -0.6786400079727173, 0.87..."


In [8]:
dataset = kg_manager.generate_huggingface_dataset(
    [vanilla_df, prompt_df, input_embeds_replace_df, input_embeds_replace_frozen_df],
    ["vanilla", "prompt", "input_embeds_replace", "input_embeds_replace_frozen_df"],
)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source_id', 'target_id', 'id_x', 'id_y', 'prompt_feature_title', 'prompt_feature_genres', 'labels', 'split', 'prompt', 'gnn_feature_(no genres listed)', 'gnn_feature_Action', 'gnn_feature_Adventure', 'gnn_feature_Animation', 'gnn_feature_Children', 'gnn_feature_Comedy', 'gnn_feature_Crime', 'gnn_feature_Documentary', 'gnn_feature_Drama', 'gnn_feature_Fantasy', 'gnn_feature_Film-Noir', 'gnn_feature_Horror', 'gnn_feature_IMAX', 'gnn_feature_Musical', 'gnn_feature_Mystery', 'gnn_feature_Romance', 'gnn_feature_Sci-Fi', 'gnn_feature_Thriller', 'gnn_feature_War', 'gnn_feature_Western', 'input_embeds_replace_source_embedding', 'input_embeds_replace_target_embedding', 'vanilla_attentions', 'vanilla_hidden_states', 'vanilla_attentions_original_shape', 'vanilla_hidden_states_original_shape', 'prompt_attentions', 'prompt_hidden_states', 'prompt_attentions_original_shape', 'prompt_hidden_states_original_shape', 'input_embeds_replace_attentions

In [10]:
dataset.save_to_disk("./data/dataset.hf")

Saving the dataset (0/10 shards):   0%|          | 0/56469 [00:00<?, ? examples/s]

Saving the dataset (0/6 shards):   0%|          | 0/34284 [00:00<?, ? examples/s]

Saving the dataset (0/6 shards):   0%|          | 0/34284 [00:00<?, ? examples/s]

In [11]:
dataset.push_to_hub("AhmadPython/MovieLens_KGE")

Uploading the dataset shards:   0%|          | 0/10 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MARS\.cache\huggingface\hub\datasets--AhmadPython--MovieLens_KGE. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/AhmadPython/MovieLens_KGE/commit/70c96f660429150e55b6437d59ee39da76891de7', commit_message='Upload dataset', commit_description='', oid='70c96f660429150e55b6437d59ee39da76891de7', pr_url=None, pr_revision=None, pr_num=None)